# Обработка пропусков в данных, кодирование категориальных признаков, масштабирование данных

План занятия:

* Обработка пропусков в данных
* Преобразование категориальных признаков в числовые
* Масштабирование и нормализация данных

Дополнительный материал:

* [Preprocessing data](https://scikit-learn.org/stable/modules/preprocessing.html)
* [Encoding Categorical Features](https://towardsdatascience.com/encoding-categorical-features-21a2651a065c)
* [Data Cleaning Challenge: Handling missing values](https://www.kaggle.com/rtatman/data-cleaning-challenge-handling-missing-values)
* [Отличия LabelEncoder и OneHotEncoder в SciKit Learn](https://habr.com/ru/post/456294/)

Для подготовки использовался [блокнот](https://github.com/ugapanyuk/ml_course_2021/blob/main/common/notebooks/missing/handling_missing_norm.ipynb) за авторством Ю.Е. Гапанюка. Также использовался материал с ресурса [blog.datalytica.ru](http://blog.datalytica.ru/2018/04/blog-post.html).

Все права сохранены за авторами блокнтотов и ресурсов.



## Проблема предобработки данных

В чем состоит проблема:

* Если в данных есть пропуски, то большинство алгоритмов машинного обучения не будут с ними работать. Даже корреляционная матрица не будет строиться корректно.
* Большинство алгоритмов машинного обучения требуют явного перекодирования категориальных признаков в числовые. Даже если алгоритм не требует этого явно, такое перекодирование возможно стоит попробовать, чтобы повысить качество модели.
* Большинство алгоритмов показывает лучшее качество на отмасштабированных признаках, в особенности алгоритмы, использующие методы градиентного спуска.

## Загрузка и первичный анализ данных



В качестве примера возьмем датасет «Automobile Data Set» с ресурса [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Automobile).

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 'Магическая' функция matplotlib
%matplotlib inline 

# Есть пять предустановленная тем Seaborn: darkgrid, whitegrid, dark, white, 
# и ticks. Каждый из них подходит для различных приложений и личных предпочтений.
sns.set(style="ticks")

Загрузим датасет:

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
names = ['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 
         'num-of-doors','body-style','drive-wheels','engine-location','wheel-base',
         'length','width','height','curb-weight','engine-type',
         'num-of-cylinders', 'engine-size','fuel-system','bore','stroke',
         'compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg',
         'price']
data = pd.read_csv(url, names=names)
print(data.shape)
data.head()

In [ ]:
data.dtypes

Количество уникальных значений для каждого столбца:

In [ ]:
data.nunique()

Было бы неплохо увидеть информацию о количестве каждого уникального значения для каждого столбца в наборе данных:

In [ ]:
for col in data.columns:
    print('{} - {}'.format(col, data[col].unique()))

Но лучше такое не делать, когда у нас много уникальных значений. Все равно все значения сходу просмотреть не удастся =)

Большинство столбцов выглядят хорошо, но есть несколько нуждающихся в очистке и нормализации. Примеры некорректных значений данных ниже.

* Строки имеют символ '?', обозначающий пропуск.
* Имеются категориальные признаки `'four' 'six' 'five' 'three' 'twelve' 'two' 'eight'`.
* `price` типа `object`.

## Замена '?' на None

In [ ]:
data = data.replace({'?' : None })
data.head()

## Приведение колонок к нужным типам

In [ ]:
data.dtypes

Видим, что колонка `"price"` имеет неправильный тип. Для исправления этого воспользуемся [`pandas.DataFrame.astype`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html#pandas-dataframe-astype)

In [ ]:
data['price'] =  pd.to_numeric(data['price'], errors='coerce')
data['peak-rpm'] =  pd.to_numeric(data['peak-rpm'], errors='coerce')
data['horsepower'] =  pd.to_numeric(data['horsepower'], errors='coerce')
data['stroke'] =  pd.to_numeric(data['stroke'], errors='coerce')
data['bore'] =  pd.to_numeric(data['bore'], errors='coerce')

data.dtypes

Итоговый результат будет таковым:

In [ ]:
data.head()



Зачастую не существует шаблонных подходов к этой задаче, поскольку подходы во многом зависит от контекста и характера данных. Например, являются ли данные случайными пропусками или же есть скрытая связь между пропусками и некоторым другим записями в обучающем примере?

Один из способов простых способов решения этой проблемы - просто игнорировать или удалять строки, в которых отсутствуют данные, выбрасывая их из нашего анализа. Однако этот метод может быть плох из-за потери информации.

Еще один способ — это заполнение пропусков, где мы заменяем отсутствующее значение каким-либо образом. Базовые реализации просто заменят все отсутствующие значения средним, медианным, либо же константой.

## Обработка пропусков в данных

### Удаление колонок, содержащих пустые значения

In [ ]:
data.isna().sum()

Удаление колонок, содержащих пустые значения:

In [ ]:
data_new_1 = data.dropna(axis=1, how='any')
(data.shape, data_new_1.shape)

In [ ]:
data_new_1.isna().sum()

### Удаление строк, содержащих пустые значения

In [ ]:
data_new_2 = data.dropna(axis=0, how='any')
(data.shape, data_new_2.shape)

In [ ]:
data_new_2.isna().sum()

### Заполнение всех пропущенных значений нулями

Заполнение всех пустых ячеек одним значением может быть некорректно, так как пустые ячейки могут содержаться в категориальных колонках.

In [ ]:
data_new_3 = data.fillna(0)
data_new_3.isnull().sum()

### "Внедрение значений" - импьютация (imputation)

#### Обработка пропусков в числовых данных

Выберем числовые колонки с пропущенными значениями. 

Сначала подготовим функцию, которая будет возвращать названия колонок датасета типа `float64` или `int64`, содержащие пустые значения:

In [ ]:
def get_columns_with_null_numbers(data):
  num_cols = []
  for col in data.columns:
    # Количество пустых значений 
    temp_null_count = data[data[col].isnull()].shape[0]
    total_count = data.shape[0]
    dt = str(data[col].dtype)
    if temp_null_count>0 and (dt=='float64' or dt=='int64'):
      num_cols.append(col)
      temp_perc = round((temp_null_count / total_count) * 100.0, 2)
      print(f'''Колонка {col}. Тип данных {dt}. Количество пустых значений {temp_null_count}, {temp_perc}%.''')
  return num_cols

Фильтр по колонкам с пропущенными значениями:

In [ ]:
data_num = data[get_columns_with_null_numbers(data)]
data_num.head()

Построем гистограмму по признакам, которые содержат пустые значения:


In [ ]:
col = get_columns_with_null_numbers(data)
for col in data_num:
    plt.hist(data[col], 50)
    plt.xlabel(col)
    plt.show()

Будем использовать встроенные [средства импьютации библиотеки scikit-learn](https://scikit-learn.org/stable/modules/impute.html).


Сначала вытащим представление колонки, которое хотим обработать:

In [ ]:
data_num_price = data_num[['price']]
data_num_price.head()

Фильтр для проверки заполнения пустых значений

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.impute import MissingIndicator

In [ ]:
indicator = MissingIndicator()
mask_missing_values_only = indicator.fit_transform(data_num_price)

In [ ]:
np.sum(mask_missing_values_only)

С помощью класса [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) можно проводить импьютацию различными показателями [центра распределения](https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D0%BA%D0%B0%D0%B7%D0%B0%D1%82%D0%B5%D0%BB%D0%B8_%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0_%D1%80%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F):

In [ ]:
strategies=['mean', 'median', 'most_frequent']

Виды [стратегий](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) внедрений значений:

* Если `mean`, то преобразование заменит отсутствующие значения, используя среднее значение по каждому столбцу. Может использоваться только с числовыми данными.
* Если `median`, то преобразование заменит отсутствующие значения, используя медиану по каждому столбцу. Может использоваться только с числовыми данными.
* Если `most_frequent`, то преобразование заменит отсутствующие значения, используя наиболее частое значение в каждом столбце. Может использоваться со строками или числовыми данными. Если таких значений несколько, возвращается только наименьшее.
* Если `constant`,  то преобразование заменит отсутствующие значения на `fill_value`. Может использоваться со строками или числовыми данными.

In [ ]:
def test_num_impute(strategy_param, column, mask_missing_values):
    imp_num = SimpleImputer(strategy=strategy_param)
    data_num_imp = imp_num.fit_transform(column)
    return data_num_imp[mask_missing_values]

Стратегия `mean`:

In [ ]:
test_num_impute(strategies[0], data_num_price, mask_missing_values_only)

Стратегия `median`:

In [ ]:
test_num_impute(strategies[1], data_num_price, mask_missing_values_only)

Стратегия `most_frequent`:

In [ ]:
test_num_impute(strategies[2], data_num_price, mask_missing_values_only)

Крассивая функция для просмотра стратегии:

In [ ]:
def test_num_impute_col(dataset, column, strategy_param):
    temp_data = dataset[[column]]
    
    indicator = MissingIndicator()
    mask_missing_values_only = indicator.fit_transform(temp_data)
    
    imp_num = SimpleImputer(strategy=strategy_param)
    data_num_imp = imp_num.fit_transform(temp_data)
    
    filled_data = data_num_imp[mask_missing_values_only]
    
    return column, strategy_param, filled_data.size, filled_data[0], filled_data[filled_data.size-1]

In [ ]:
data[['horsepower']].describe()

In [ ]:
test_num_impute_col(data, 'horsepower', strategies[0])

In [ ]:
test_num_impute_col(data, 'horsepower', strategies[1])

In [ ]:
test_num_impute_col(data, 'horsepower', strategies[2])

#### Обработка пропусков в категориальных данных

Выберем категориальные колонки с пропущенными значениями. 

Сначала подготовим функцию, которая будет возвращать названия колонок датасета типа `object`, содержащие пустые значения:

In [ ]:
def get_columns_with_null_strings(data):
  total_count = data.shape[0]
  cat_cols = []
  for col in data.columns:
    # Количество пустых значений 
    temp_null_count = data[data[col].isnull()].shape[0]
    dt = str(data[col].dtype)
    if temp_null_count>0 and (dt=='object'):
      cat_cols.append(col)
      temp_perc = round((temp_null_count / total_count) * 100.0, 2)
      print(f'''Колонка {col}. Тип данных {dt}. Количество пустых значений {temp_null_count}, {temp_perc}%.''')
  return cat_cols

In [ ]:
cat_cols = get_columns_with_null_strings(data)
cat_cols

Класс SimpleImputer можно использовать для категориальных признаков со стратегиями `most_frequent` или `constant`.

In [ ]:
cat_temp_data = data[['num-of-doors']]
cat_temp_data.head()

In [ ]:
data['num-of-doors'].unique()

In [ ]:
cat_temp_data[cat_temp_data['num-of-doors'].isnull()].shape

Импьютация наиболее частыми значениями:

In [ ]:
imp2 = SimpleImputer(missing_values=None, strategy='most_frequent')
data_imp2 = imp2.fit_transform(cat_temp_data)
np.unique(data_imp2)

Импьютация константой:

In [ ]:
imp3 = SimpleImputer(missing_values=None, strategy='constant', fill_value='NA')
data_imp3 = imp3.fit_transform(cat_temp_data)
np.unique(data_imp3) # Пустые значения отсутствуют

## Преобразование категориальных признаков в числовые



Многие алгоритмы машинного обучения ожидают числовые входные данные, поэтому нам нужно выяснить способ представления наших категориальных данных численным образом.




### Кодирование категорий целочисленными значениями - label encoding



Для кодирования категориальных признаков мы также можем воспользоваться произвольным присвоением числового значения для каждой категории и отображение набора данных из исходных категорий в каждое соответствующее число.

In [ ]:
data = data.replace({'eight':8,'five':5,'four':4,'six':6, 'three':3, 'twelve':12, 'two':2})
data.head()

In [ ]:
data['num-of-doors'].unique()

Этот процесс известен как `Label Encoding` и `sklearn` может сделать это за нас. Воспользуемся [`sklearn.preprocessing.LabelEncoder`](https://scikit-learn.org/0.21/modules/generated/sklearn.preprocessing.LabelEncoder.html):

In [ ]:
from sklearn.preprocessing import LabelEncoder

LebEncoders = {'make' : LabelEncoder(), 'fuel-system' : LabelEncoder(),
               'body-style' : LabelEncoder(), 'fuel-type' : LabelEncoder(),
               'aspiration' : LabelEncoder(), 'drive-wheels' : LabelEncoder(),
               'engine-location' : LabelEncoder(), 'engine-type': LabelEncoder()
               }

# LabelEncoder может использоваться для нормализации меток (кодирование меток со значением от 0 до n_classes-1)
col_le_make = LebEncoders['make'].fit_transform(data['make']) # взять исходные метки и вернуть зашифрованные надписи
np.unique(col_le_make)

In [ ]:
col_le_make[:10]

In [ ]:
# преобразовать метки обратно в оригинальную кодировку
LebEncoders['make'].inverse_transform(col_le_make)[:10]

In [ ]:
data_new = data.copy()

# Заменяем столбцы нашим энкодером
for col in LebEncoders:
    data_new[col] = LebEncoders[col].fit_transform(data[col])

data_new.head()

### Кодирование категорий наборами бинарных значений - one-hot encoding

в зависимости от имеющихся у нас данных мы можем столкнуться с ситуацией, когда после кодирования признаков наша модель запутается, ложно предположив, что данные связаны порядком или иерархией, которого на самом деле нет. Чтобы этого избежать мы воспользуемся `OneHotEncoder`.


Этот кодировщик берёт столбец с категориальными данными, который был предварительно закодирован в признак, и создаёт для него несколько новых столбцов. Числа заменяются на единицы и нули, в зависимости от того, какому столбцу какое значение присуще.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# взять исходные метки и вернуть зашифрованные надписи
ohe = OneHotEncoder()
col_oh_make = ohe.fit_transform(data[['make']]) 
col_oh_make

In [ ]:
col_oh_make.shape

In [ ]:
col_oh_make.todense()[0:10]

Быстрый вариант one-hot кодирования через метод [`pandas.get_dummies`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html): 

In [ ]:
pd.get_dummies(data[['make']]).head()

Добавить столбец для указания `NaN`:

In [ ]:
pd.get_dummies(data[['make']], dummy_na=True).head()

## Масштабирование и нормализация данных

Термины "масштабирование" и "нормализация" часто используются как синонимы. Масштабирование предполагает изменение диапазона измерения величины, а нормализация - изменение распределения этой величины.

Подробнее можно ознакомиться по ссылкам:

* [Блокнот](https://github.com/ugapanyuk/ml_course_2021/blob/main/common/notebooks/missing/handling_missing_norm.ipynb) Ю.Е. Гапанюка
* [Что такое Scikit Learn - гайд по популярной библиотеке Python для начинающих](https://datastart.ru/blog/read/chto-takoe-scikit-learn-gayd-po-populyarnoy-biblioteke-python-dlya-nachinayuschih)
* [Standardization, or mean removal and variance scaling](https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling)
* [2 простых способа нормализовать данные в Python](https://dev-gang.ru/article/-prostyh-sposoba-normalizovat-dannye-v-python-7qqrhmlppl/)